In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas as pd

In [ ]:
!pip install ibm_db

     |████████████████████████████████| 727kB 4.7MB/s 
  Created wheel for ibm-db: filename=ibm_db-3.0.2-cp36-cp36m-linux_x86_64.whl size=38977337 sha256=686910a76164ed4e868ac6c02c49a9aa57db1347a7e62a7d90c9b5bd92ce9522
  Stored in directory: /root/.cache/pip/wheels/84/64/42/10b217b545e8085b72160632c61bfa0909f661dad89326a956
Successfully built ibm-db


In [ ]:
import ibm_db

In [ ]:
dsn_hostname = "dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "bmt70440"        # e.g. "abc12345"
dsn_pwd = "k54tm9n1rbnmb@dd"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"   

In [ ]:
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=bmt70440;PWD=k54tm9n1rbnmb@dd;


In [ ]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

Connected to database:  BLUDB as user:  bmt70440 on host:  dashdb-txn-sbox-yp-lon02-04.services.eu-gb.bluemix.net


In [ ]:
import pandas as pd
import ibm_db_dbi

In [ ]:
pconn1 = ibm_db_dbi.Connection(conn)

In [ ]:
selectQuery1 = "select count(*) from CRIME"
df1 = pd.read_sql(selectQuery1, pconn1)

In [ ]:
#Find the total number of crimes recorded in the crime table
np.array(df1)

array([[533.]])

In [ ]:
selectQuery2 = "select * from CRIME LIMIT 10"
df2 = pd.read_sql(selectQuery2, pconn1)

In [ ]:
# Retrieve first 10 rows from the CRIME table
df2

,ID,CASE_NUMBER,BLOCK,IUCR,PRIMARY_TYPE,DESCRIPTION,LOCATION_DESCRIPTION,ARREST,DOMESTIC,BEAT,DISTRICT,WARD,COMMUNITY_AREA_NUMBER,FBICODE,X_COORDINATE,Y_COORDINATE,YEAR,LATITUDE,LONGITUDE,LOCATION
0,3512276,HK587712,047XX S KEDZIE AVE,890,THEFT,FROM BUILDING,SMALL RETAIL STORE,FALSE,FALSE,911,9,14,58,6,1155838,1873050,2004,41.807440,-87.703956,"(41.8074405, -87.703955849)"
1,3406613,HK456306,009XX N CENTRAL PARK AVE,820,THEFT,$500 AND UNDER,OTHER,FALSE,FALSE,1112,11,27,23,6,1152206,1906127,2004,41.898280,-87.716406,"(41.898279962, -87.716405505)"
2,8002131,HT233595,043XX S WABASH AVE,820,THEFT,$500 AND UNDER,NURSING HOME/RETIREMENT HOME,FALSE,FALSE,221,2,3,38,6,1177436,1876313,2011,41.815933,-87.624642,"(41.815933131, -87.624642127)"
3,7903289,HT133522,083XX S KINGSTON AVE,840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,FALSE,FALSE,423,4,7,46,6,1194622,1850125,2010,41.743665,-87.562463,"(41.743665322, -87.562462756)"
4,10402076,HZ138551,033XX W 66TH ST,820,THEFT,$500 AND UNDER,ALLEY,FALSE,FALSE,831,8,15,66,6,1155240,1860661,2016,41.773455,-87.706480,"(41.773455295, -87.706480471)"
5,7732712,HS540106,006XX W CHICAGO AVE,810,THEFT,OVER $500,PARKING LOT/GARAGE(NON.RESID.),FALSE,FALSE,1323,12,27,24,6,1171668,1905607,2010,41.896447,-87.644939,"(41.896446772, -87.644938678)"
6,10769475,HZ534771,050XX N KEDZIE AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,1713,17,33,14,6,1154133,1933314,2016,41.972845,-87.708600,"(41.972844913, -87.708600079)"
7,4494340,HL793243,005XX E PERSHING RD,860,THEFT,RETAIL THEFT,GROCERY FOOD STORE,TRUE,FALSE,213,2,3,38,6,1180448,1879234,2005,41.823880,-87.613504,"(41.823879885, -87.613503857)"
8,3778925,HL149610,100XX S WASHTENAW AVE,810,THEFT,OVER $500,STREET,FALSE,FALSE,2211,22,19,72,6,1160129,1838040,2005,41.711281,-87.689179,"(41.711280513, -87.689179097)"
9,3324217,HK361551,033XX W BELMONT AVE,820,THEFT,$500 AND UNDER,SMALL RETAIL STORE,FALSE,FALSE,1733,17,35,21,6,1153590,1921084,2004,41.939296,-87.710923,"(41.939295821, -87.710923442)"


In [ ]:
selectQuery3 = "select count(*) from CRIME where ARREST = TRUE"
df3 = pd.read_sql(selectQuery3, pconn1)

In [ ]:
#How many crimes involve an arrest
np.array(df3)

array([[163.]])

In [ ]:
selectQuery4 = "select distinct PRIMARY_TYPE from CRIME where LOCATION_DESCRIPTION = 'GAS STATION'"
df4 = pd.read_sql(selectQuery4, pconn1)

In [ ]:
# Which unique types of crimes have been recorded at a GAS STATION?
df4

,PRIMARY_TYPE
0,CRIMINAL TRESPASS
1,NARCOTICS
2,ROBBERY
3,THEFT


In [ ]:
selectQuery5 = "select COMMUNITY_AREA_NAME from CENSUS where COMMUNITY_AREA_NAME like 'B%'"
df5 = pd.read_sql(selectQuery5, pconn1)

In [ ]:
#In the CENUS_DATA table list all community areas whose names start with the letter ‘B’
df5

,COMMUNITY_AREA_NAME
0,Belmont Cragin
1,Burnside
2,Brighton Park
3,Bridgeport
4,Beverly


In [ ]:
selectQuery6 = "select NAME_OF_SCHOOL from \
(select NAME_OF_SCHOOL, HEALTHY_SCHOOL_CERTIFIED, COMMUNITY_AREA_NUMBER from PUBLICSCHOOL where COMMUNITY_AREA_NUMBER between 10 and 15 and HEALTHY_SCHOOL_CERTIFIED = 'Yes')"

df6 = pd.read_sql(selectQuery7, pconn1)

In [ ]:
#List the schools in Community Areas 10 to 15 that are healthy school certified?
df6

,NAME_OF_SCHOOL
0,Rufus M Hitch Elementary School


In [ ]:
selectQuery7 = "select avg(SAFETY_SCORE) from PUBLICSCHOOL"
df7 = pd.read_sql(selectQuery7, pconn1)

In [ ]:
#What is the average school Safety Score?
np.array(df7)

array([[49.504873]])

In [ ]:
selectQuery = "select COMMUNITY_AREA_NAME, COLLEGE_ENROLLMENT as number_of_students from PUBLICSCHOOL \
order by number_of_students desc fetch first 5 rows only"
df= pd.read_sql(selectQuery, pconn1)

In [ ]:
df

,COMMUNITY_AREA_NAME,NUMBER_OF_STUDENTS
0,NORTH CENTER,4368
1,ARCHER HEIGHTS,3320
2,NORWOOD PARK,2922
3,BRIGHTON PARK,2883
4,IRVING PARK,2366


In [ ]:
selectQuery8 = "select COMMUNITY_AREA_NAME from CENSUS \
where COMMUNITY_AREA_NUMBER = \
(select COMMUNITY_AREA_NUMBER from PUBLICSCHOOL order by SAFETY_SCORE limit 1)"
df8 = pd.read_sql(selectQuery8, pconn1)

In [ ]:
# Use a sub-query to determine which Community Area has the least value for Safety Score?
df8

,COMMUNITY_AREA_NAME
0,Washington Park


In [ ]:
selectQuery9 = "select PER_CAPITA_INCOME, COMMUNITY_AREA_NAME from CENSUS \
where COMMUNITY_AREA_NUMBER = \
(select COMMUNITY_AREA_NUMBER from PUBLICSCHOOL \
where SAFETY_SCORE = 1)"
df9 = pd.read_sql(selectQuery9, pconn1)

In [ ]:
#Find the Per Capita Income of the Community Area which has a school Safety Score of 1
df9

,PER_CAPITA_INCOME,COMMUNITY_AREA_NAME
0,13785,Washington Park


In [ ]:
selectQuery8 = "select COMMUNITY_AREA_NAME , COLLEGE_ENROLLMENT from PUBLICSCHOOL order by COLLEGE_ENROLLMENT desc LIMIT 5 "
df8 = pd.read_sql(selectQuery, pconn1)

In [ ]:
#List the top 5 Community Areas by average College Enrollments [number of students]
df8

,COMMUNITY_AREA_NAME,COLLEGE_ENROLLMENT
0,NORTH CENTER,4368
1,ARCHER HEIGHTS,3320
2,NORWOOD PARK,2922
3,BRIGHTON PARK,2883
4,IRVING PARK,2366
